In [29]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
df = pd.read_excel('combined_excel.xlsx',index_col=0)[['45A','SPEC']]
print(df.shape)
df.head(3)

(3643, 2)


,45A,SPEC
3,"MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...",MASS PVC RESIN B-57
4,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,(鄰苯二甲酐)\nPHTHALIC ANHYDRIDE
5,+ COMMODITY: LLDPE TAISOX 3470+ QUANTITY: 32.0...,LINEAR LOW DENSITY\nPOLYETHYLENE RESIN\nTAISOX...


# 去除不必要的東西

In [30]:
import re

def preprocess(x):
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 3.去除換行符號
    x = str.strip(x) # 4.移除左右空白
    return x

df['45A'] = df['45A'].apply(preprocess)
df['SPEC'] = df['SPEC'].apply(preprocess)
df.loc[:,['45A','SPEC']].head()

,45A,SPEC
3,MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,MASS PVC RESIN B-57
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2FC...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470
7,COMMODITY ...,POLYESTER PARTIALYORIENTED YARN
9,COMMODITY ...,POLYESTER FILAMENT YARN


# 寶典標註

In [31]:
import numpy as np

寶典 = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx')
產品集合 = set(寶典['品名'].values)
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:# 在寶典中搜索
            if p in df.loc[i,'45A']: 
                products.append(p)
        try:
            labels[i] = max(products,key=len) # 選最長的產品
        except:
            labels[i] = np.nan # 找不到就算了
    predict = pd.DataFrame(index=labels.keys(),columns=['Baodian_predict'])
    predict['Baodian_predict'] = labels.values()
    return predict
predict = Collection_method(df,產品集合)
df = df.join(predict)
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/3643 [00:00<?, ?it/s]

,45A,SPEC,Baodian_predict
3,MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,MASS PVC RESIN B-57,PVC RESIN B-57
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE PA
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2FC...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470,LLDPE TAISOX
7,COMMODITY ...,POLYESTER PARTIALYORIENTED YARN,POLYESTER PARTIALY ORIENTED YARN
9,COMMODITY ...,POLYESTER FILAMENT YARN,POLYESTER PARTIALY ORIENTED YARN
...,...,...,...
8403,COMMODITY MONO ETHYLENE GLYCOLx000D...,MONO ETHYLENE GLYCOL,MONO ETHYLENE GLYCOL
8413,CFR KOBE JAPANx000DVISCOSE RAYON STAPLE FIBERx...,VISCOSE RAYON STAPLE FIBER15DX40MM SEMI DULL,VISCOSE RAYON STAPLE FIBER
8414,DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDM...,PC RESIN GRADE NOAC2916,PC RESIN
8415,DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDM...,PC RESIN GRADE NOAC2820-A320,PC RESIN


# 製作Y_label

In [32]:
ok_ = 0
no_ = 0
y_label = []

for i in tqdm(range(len(df))):
    # 1.判斷SPEC是否匹配
    if str(df['SPEC'].values[i]) in str(df['45A'].values[i]):
        y_label.append(df['SPEC'].values[i])
        ok_ += 1
        continue #continue：強制跳出 ❮本次❯ 迴圈，繼續進入下一圈
    
    # 2.如果SPEC無法匹配到則可以判斷寶典是否匹配
    if str(df['Baodian_predict'].values[i]) in str(df['45A'].values[i]):
        y_label.append(df['Baodian_predict'].values[i])
        ok_ += 1
        continue #continue：強制跳出 ❮本次❯ 迴圈，繼續進入下一圈
    
    # 3.如果都不匹配 則會觸發以下程序
    no_ += 1
    y_label.append(np.nan)

print(ok_,no_)
df['Y_label'] = y_label
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3643 [00:00<?, ?it/s]

3618 25


,45A,SPEC,Baodian_predict,Y_label
3,MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,MASS PVC RESIN B-57,PVC RESIN B-57,MASS PVC RESIN B-57
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE PA,PHTHALIC ANHYDRIDE
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2FC...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470,LLDPE TAISOX,LLDPE TAISOX
7,COMMODITY ...,POLYESTER PARTIALYORIENTED YARN,POLYESTER PARTIALY ORIENTED YARN,POLYESTER PARTIALY ORIENTED YARN
9,COMMODITY ...,POLYESTER FILAMENT YARN,POLYESTER PARTIALY ORIENTED YARN,POLYESTER FILAMENT YARN


# 斷詞模型

In [33]:
import wordninja
from wordninja import LanguageModel
import re

寶典 = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx')
產品集合 = list(set(寶典['品名'].values))
產品集合 = [str(i).lower() for i in 產品集合]
print(產品集合[:5])

with open('wordninja_words.txt',encoding="utf-8") as f:
    wordninja_words_lst = f.read().split('\n')
print(wordninja_words_lst[:5])

SPEC集合 = df['Y_label'].values.tolist()
SPEC集合 = [str(i).lower() for i in SPEC集合]
print(SPEC集合[:5])

全部集合 = list(set(產品集合+SPEC集合+wordninja_words_lst))
print(全部集合[:5])

with open('全部集合.txt', 'w',encoding="utf-8") as f:
    lines = [i + '\n' for i in 全部集合]
    f.writelines(lines)
import gzip
import shutil
with open('全部集合.txt', 'rb') as f_in, gzip.open('全部集合.txt.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

class LanguageModel2(LanguageModel):
    def split(self, s):
        _SPLIT_RE = re.compile("[^a-zA-Z0-9-']+")
        l = [self._split(x) for x in _SPLIT_RE.split(s)]
        return [item for sublist in l for item in sublist]

lm = LanguageModel2('全部集合.txt.gz')
' '.join(lm.split(df['45A'].values[0]))

['epichlorohydrine ech', 'lda33xmr5x', 'spun viscose yarn', 'copolymer pvc c-15', 'acrylate emulsion']
['the', 'of', 'in', 'a', 'and']
['mass pvc resin b-57', 'phthalic anhydride', 'lldpe taisox', 'polyester partialy oriented yarn', 'polyester filament yarn']
['', 'niederwil', 'paltriness', 'mirrored', 'quisquam']


'MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US DM TAS PER PROFORMA INVOICE NO A 0 B 002 - 11 D A T E D 10 - 12 - 2020 C I F PIPA VAV PORT INDIA IN COTE RMS 2010'

# 斷詞處理

In [34]:
lst = []
for i in tqdm(range(len(df))):
    lst.append(' '.join(lm.split(df['45A'].values[i])))
df['45A'] = lst
df[['45A','Y_label']].head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3643 [00:00<?, ?it/s]

,45A,Y_label
3,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57
4,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX
7,COMMODITY AMOUNT POLYESTER TEXTURED YAR NHS CO...,POLYESTER PARTIALY ORIENTED YARN
9,COMMODITY AMOUNT POLYESTER TEXTURED YAR NHS CO...,POLYESTER FILAMENT YARN


# 清洗

In [35]:
df = df.dropna(axis=0).reset_index(drop=True)
print(df.shape)
keep_lst = []
for i in range(len(df)):
    if df.iloc[i]['Y_label'] in df.iloc[i]['45A']:
        keep_lst.append(i)
df = df.iloc[keep_lst]
print(df.shape)

(3614, 4)
(3198, 4)


# 起始結束標註

In [36]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['45A'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df[['45A','Y_label','string_Y_1','string_Y_2']].head()

,45A,Y_label,string_Y_1,string_Y_2
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,0,19
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,0,18
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,10,22
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,7,23
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,7,23


In [37]:
print(df.shape)

(3198, 6)


# 保存

In [38]:
df.to_csv('preprocess_for_SQUAD_wordninja.csv')